# Data visualization Project

## Part 3

In [65]:
#import the essential packages
import ipywidgets as widgets
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, HBox, VBox
import numpy as np
from bqplot import *
from bqplot import Tooltip, pyplot, marks
import pandas as pd
import us
from IPython.display import display

import traitlets

#little bit of magic!
%matplotlib inline

In [2]:
names = ["date", "city", "state", "country", "shape", "duration_seconds", "duration_reported", "description", "report_date", "latitude", "longitude"]

ufo = pd.read_csv("ufo-scrubbed-geocoded-time-standardized.csv",
                  names = names, parse_dates = ["date", "report_date"])

abbr_to_fips = us.states.mapping('abbr', 'fips')
ufo["fips"] = ufo["state"].apply(lambda a: int(abbr_to_fips.get(str(a).upper(), -1)))

total_time = ufo.groupby(["fips"])["duration_seconds"].sum()

total_sightings = ufo.groupby(["fips"])["report_date"].count()

In [3]:
df = pd.DataFrame([total_sightings, total_time])
df = df.T
df.columns = ['total sightings', 'total time']

df = df[1:]
df
df['total sightings']

fips
1      691.0
2      354.0
4     2689.0
5      666.0
6     9655.0
8     1505.0
9      968.0
10     183.0
11      99.0
12    4200.0
13    1347.0
15     353.0
16     554.0
17    2645.0
18    1386.0
19     707.0
20     653.0
21     914.0
22     598.0
23     633.0
24     911.0
25    1358.0
26    2071.0
27    1081.0
28     415.0
29    1576.0
30     510.0
31     405.0
32     905.0
33     535.0
34    1512.0
35     815.0
36    3219.0
37    1869.0
38     138.0
39    2425.0
40     766.0
41    1845.0
42    2582.0
44     290.0
45    1076.0
46     196.0
47    1193.0
48    3677.0
49     743.0
50     307.0
51    1393.0
53    4268.0
54     486.0
55    1333.0
56     205.0
72      33.0
Name: total sightings, dtype: float64

In [4]:
type(total_sightings)

pandas.core.series.Series

In [32]:
ufo['year'] = pd.DatetimeIndex(ufo['date']).year

In [79]:

def showPlot(change):
    global ch
    print(type(change['new']))


    if len(change.new) > 1:
        states_map.selected = [change.new[-1]]
    elif len(change.new) == 1:
        print("yay")
    
        ch = states_map.selected[-1:]
        ufo_state = ufo[ufo['fips'] == ch[0]]

        print(states_map.selected[-1:])
        scatter_plot = interact(getCorresPlot, year = widgets.IntRangeSlider(
                    value=[ufo_state['year'].max() - 50,ufo_state['year'].max()],
                    min=ufo_state['year'].min(),
                    max=ufo_state['year'].max(),
                    continuous_update=False,
                    step=10,
                ));
        


def getCorresPlot(year):   
    
    print(year[0])
    print(year[1])
    print(ch)
    ufo_state = ufo[ufo['fips'] == ch[0]]
    #ch[:] = []
    ufo_plot = ufo_state[ufo_state['year'] > year[0]]
    
    total_time_year = ufo_state.groupby(["year"])["duration_seconds"].sum() 
    total_sightings_year = ufo_state.groupby(["year"])["report_date"].count()
    
    sc_x = LinearScale()
    sc_y = LinearScale()
    scatter = Scatter(x=ufo_plot['year'].unique(), y=total_sightings_year, scales={'x': sc_x, 'y': sc_y}, colors=['blue'])

    ax_x = Axis(label='Year', scale=sc_x)
    ax_y = Axis(label='Test Y', scale=sc_y, orientation='vertical', tick_format='0.2f')

    fig = Figure(axes=[ax_x, ax_y], marks=[scatter])
    
    return fig
    

        

def getMap(mode):
    if mode == 'total sightings':
        def_tt = Tooltip(fields=['name', 'color'], labels = ['state', 'total_sightings_year'])
    else:
        def_tt = Tooltip(fields=['name', 'color'], labels = ['state', 'total_time_year'])
        
    global states_map
    
    map_styles = {'scales': {'projection': bqplot.AlbersUSA(),
                             'color': bqplot.ColorScale(colors=["brown", "blue", "red"])},
                  'color': df[mode].to_dict()}
    states_map = bqplot.Map(map_data=bqplot.topo_load('map_data/USStatesMap.json'), **map_styles, tooltip=def_tt
                           )

    states_map.interactions = {'click': 'select', 'hover': 'tooltip'}
    #states_map.observe(getCorresPlot)
    #display(getCorresPlot)
    
    #states_map.observe(showPlot, names= ['selected'])
    map_figure = bqplot.Figure(marks=[states_map], fig_margin={'top':30, 'bottom':30, 'left':0, 'right':0})
    return map_figure


map_figure = interact(getMap, mode = widgets.Dropdown(
                                options=['total sightings', 'total time'],
                                value='total sightings',
                                disabled=False,
                            ));






states_map.observe(showPlot, names= ['selected'])

interactive(children=(Dropdown(description='mode', options=('total sightings', 'total time'), value='total sightings'), Output()), _dom_classes=('widget-interact',))

In [63]:
w = widgets.Dropdown(
        options=['Total number of Sightings', 'Total Time in sightings'],
        value='Total number of Sightings',
        disabled=False,
    )

toMapDict = {}
def onChange(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] == 'Total Time in sightings':
            toMap = total_time
        else:
            toMap = total_sightings
        toMapDict = toMap.to_dict()
            
        print('toMap is ', toMap.to_dict())
        def_tt = Tooltip(fields=['id', 'name'])    
        map_styles = {'scales': {'projection': bqplot.AlbersUSA(),
                                 'color': bqplot.ColorScale(colors=["red", "blue"])},
                      'color': toMapDict}
        states_map = bqplot.Map(map_data=bqplot.topo_load('map_data/USStatesMap.json'), **map_styles, tooltip=def_tt)

        states_map.interactions = {'click': 'select', 'hover': 'tooltip'}
        map_fig = bqplot.Figure(marks=[states_map])
        toMapDict.clear()
        map_fig
            

def doObserve():
    w.observe(onChange)
    display(map_fig)
    type(display)
    
doObserve()


NameError: name 'map_fig' is not defined

In [76]:
ufo_state = ufo[ufo['fips'] == [30]]
ufo_state

,date,city,state,country,shape,duration_seconds,duration_reported,description,report_date,latitude,longitude,fips,year
602,1993-10-01 20:00:00,pryor,mt,us,light,1800.0,20-30 minutes,Floating Light&#44 Like a Spotlight,2005-11-03,45.429722,-108.532500,30,1993
1014,1999-10-12 01:30:00,missoula,mt,us,triangle,60.0,1 minute,Some of my friend and I were outside&#44 smoki...,1999-11-23,46.872222,-113.993056,30,1999
1154,2011-10-12 21:00:00,zortman,mt,us,light,1800.0,30 mins,Small bright yellow white light forming into t...,2011-10-19,47.917778,-108.525556,30,2011
1365,2008-10-13 21:30:00,butte,mt,us,NaN,3600.0,1 hour,40-50 lights flying over Butte Montana in a V ...,2008-10-31,46.003889,-112.533889,30,2008
1493,1995-10-14 22:30:00,troy,mt,us,NaN,2.0,2 sec.,Two women witness green obj&#44 &quot;size of ...,1999-11-02,48.463333,-115.888611,30,1995
1580,2005-10-14 19:45:00,lincoln (east of),mt,us,egg,1800.0,15-30 minutes,egg/circle shaped&#44 red/yellow/white lights ...,2005-10-20,46.955000,-112.680833,30,2005
1781,1975-10-15 22:00:00,ryegate (south of),mt,us,circle,3600.0,1 hour,10/15/75. Round shape&#44 extremely bright whi...,2003-10-31,46.297222,-109.258056,30,1975
1855,1987-10-15 22:30:00,baker,mt,us,light,600.0,5 to 10 min,bright light came out from behind a cloud and ...,2005-05-24,46.366944,-104.284167,30,1987
1891,1993-10-15 00:00:00,butte,mt,us,sphere,30.0,30 secs,A hovering object in the vicinity of &quot;The...,2003-10-31,46.003889,-112.533889,30,1993
2059,2002-10-15 20:10:00,rexford,mt,us,chevron,30.0,30 sec,Twelve chevron shaped crafts spotte over Lake ...,2002-10-28,48.899444,-115.171111,30,2002


NameError: name 'state' is not defined

In [58]:
ch = [0,1,2,3]
ch

[0, 1, 2, 3]

In [59]:
del ch[:]
ch

[]